<a href="https://colab.research.google.com/github/aliahalotaibi73/week6_exercises/blob/main/Motion_Detection_exercise_Aliah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motion Detection Using Background Subtraction in OpenCV

### Introduction

In this tutorial, we will learn how to perform **motion detection** using background subtraction in OpenCV. Background subtraction is a technique where we analyze the difference between the current frame and a background model to detect objects that have moved in the scene.

### Step 1: Importing Required Libraries

In [1]:
import numpy as np
import cv2


### Step 2: Setting Up Video Capture

In [2]:

cap = cv2.VideoCapture('/content/Motion_Detection_Test.mp4')
print(cap.isOpened())

True


Here, we initialize the video capture object by loading a video file (`Motion_Detection_Test.mp4`). This object allows us to read frames from the video in a loop.

### Step 3: Defining the Video Writer to Save the Output

In [3]:

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('motion_output.mp4', fourcc, 20.0, (600, 500))


In this step, we create a `VideoWriter` object to save the processed frames with motion detection. The codec is set to `'mp4v'`, and the output file is `motion_output.mp4`. The resolution is `(600x500)` with a frame rate of `20.0`.

### Step 4: Creating the Background Subtractor

In [4]:

# Create a background subtractor using MOG2
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)


We use the MOG2 background subtractor provided by OpenCV. This method detects moving objects by subtracting the background. The `detectShadows=True` option helps distinguish moving objects from their shadows, improving accuracy.

### Step 5: Processing the Video Frame by Frame

In [12]:

while True:
    # Capture each frame of the video
    success, img = cap.read()

    # Check if the frame is captured successfully
    if success:
        # Resize the frame for consistency
        img = cv2.resize(img, (600, 500))

        # Apply the background subtractor to detect moving objects
        fgmask = fgbg.apply(img)

        # Create a binary thresholded image for better motion detection
        _, thresh = cv2.threshold(fgmask.copy(), 180, 255, cv2.THRESH_BINARY) # less than 180 will be black and above will be wight

        # Define a kernel for morphological operations, kernel means filters
        kernel = np.ones((7, 7), np.uint8)

        # Apply erosion to remove noise from the thresholded image
        thresh = cv2.erode(thresh, kernel)

        # Apply dilation to strengthen the detected moving objects
        thresh = cv2.dilate(thresh, None, iterations=6)

        # Find contours of the detected motion
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # شكل الكنتور لازم يكون انسيابي وناعم بدون ذبذبات

        # Draw rectangles around detected motion
        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)

            # Only consider significant motion (area > 1200)
            if area > 1200:
                # Get the bounding box coordinates for the motion
                x, y, w, h = cv2.boundingRect(contour)

                # Draw a rectangle around the detected motion and label it
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3) # dry rectangle
                cv2.putText(img, 'MOTION DETECTED', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Write the processed frame with motion detection to the output video
                out.write(img)

    else:
        # Break the loop if no more frames are available
        break


In this step, we continuously capture frames from the video, apply the background subtractor, and use contour detection to identify significant motion. Rectangles are drawn around detected motion, and each frame is saved to the output video.

### Step 6: Releasing Resources

In [13]:

# Release the video capture and writer objects
cap.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


After processing all the frames, we release the resources by closing the video capture and writer objects. All OpenCV windows are also closed.
[Source](https://github.com/zain18jan2000/Motion-Detection-Using-BackgroundSubtractorMOG2/tree/main) for this tutorial.